In [ ]:
## Importing libraries, classes, and functions

from bmi_topography import Topography
import matplotlib
import numpy as numpy
from landlab import RasterModelGrid
from landlab.components import  (
    DepressionFinderAndRouter,
    FlowAccumulator,
    FlowDirectorD8, FlowDirectorMFD,
    FlowDirectorSteepest,
)
from landlab.plot.drainage_plot import drainage_plot
import matplotlib.pyplot as plt
from matplotlib import cm
def surf_plot(mg, surface="topographic__elevation", title="Surface plot of topography"):
    plt.figure()
    ax = plt.axes(projection="3d")

    # Plot the surface.
    Z = mg.at_node[surface].reshape(mg.shape)
    color = cm.gray((Z - Z.min()) / (Z.max() - Z.min()))
    ax.plot_surface(
        mg.x_of_node.reshape(mg.shape),
        mg.y_of_node.reshape(mg.shape),
        Z,
        rstride=1,
        cstride=1,
        facecolors=color,
        linewidth=0.0,
        antialiased=False,
    )

In [ ]:
#help(Topography)

In [ ]:
# Defininf raster bounds
topo = Topography(
    dem_type="SRTMGL1",
    south=-1.540602,
    north=-1.424255,
    west=-78.9056896,
    east=-78.753253,
    output_format="GTiff",
    cache_dir="."
)

In [ ]:
# Clipping the raster online
fname = topo.fetch()
print(fname)

In [ ]:
# Downloading it to the console
da = topo.load()
print(da)

In [ ]:
# Plot DEM with coordinates
da.plot()

In [ ]:
# Creater grid for flow direction
mg = RasterModelGrid((da.shape[1:3]),xy_spacing=(30,30)) # dimensions m^2
mg.set_closed_boundaries_at_grid_edges(True, True, True, False)
da = da.astype(numpy.float64) # type casting to float64, float32 will not work
mg.add_field("topographic__elevation", numpy.flip(da,1), at="node"); # don't forget to flip image, because of ASCII indeces

In [ ]:
# Plot grid in 3D figure
surf_plot(mg, title="Elevation [masl]")

In [ ]:
# Plot grind in 2D figure
mg.imshow("topographic__elevation")

In [ ]:
fdMFD = FlowDirectorMFD(mg, diagonal = False)
for _ in range(200):
    fdMFD.run_one_step()

In [ ]:
plt.figure()
drainage_plot(mg,title="Flow Director MFD")

In [ ]:
# Creater grid for flow direction
mg2 = RasterModelGrid((da.shape[1:3]),xy_spacing=(30,30)) # dimensions m^2
mg2.set_closed_boundaries_at_grid_edges(True, True, True, False)
mg2.add_field("topographic__elevation", numpy.flip(da,1), at="node"); # don't forget to flip image, because of ASCII indeces

In [ ]:
fdSteep = FlowDirectorSteepest(mg2)
for _ in range(200):
    fdSteep.run_one_step()
plt.figure()
drainage_plot(mg2,title="Flow Director Steepest")

In [ ]:
# Creater grid for flow direction
mg3 = RasterModelGrid((da.shape[1:3]),xy_spacing=(30,30)) # dimensions m^2
mg3.set_closed_boundaries_at_grid_edges(True, True, True, False)
mg3.add_field("topographic__elevation", numpy.flip(da,1), at="node"); # don't forget to flip image, because of ASCII indeces

In [ ]:
fdD8 = FlowDirectorD8(mg3)
for _ in range(200):
    fdD8.run_one_step()
plt.figure()
drainage_plot(mg3,title="Flow Director D8")